# HOW MUCH ENERGY WILL A BUILDING CONSUME?

### BUSINESS QUESTIONS?

##### 1. Which meter types contribute the most to energy consumption? Which saves the most energy?
##### 2. What are the factors that contribute to energy consumption? 
##### 3. Which time of the day or month of the year is energy consumed the most or saved the most? 

In [74]:
#import the neccesary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)

In [75]:
#load in the data
weather=pd.read_csv('weather_train.csv')
data=pd.read_csv('train.csv')
building=pd.read_csv('building_metadata.csv')

##  Data Cleaning

In [76]:
weather.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [78]:
building.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [79]:
print(data.shape)
print(building.shape)
print(weather.shape)

(20216100, 4)
(1449, 6)
(139773, 9)


## Missing data

In [80]:
#check for missing data in the data dataset
data.isnull().sum()

building_id      0
meter            0
timestamp        0
meter_reading    0
dtype: int64

In [81]:
#check for missing data in the building dataset
building.isnull().sum()

site_id           0
building_id       0
primary_use       0
square_feet       0
year_built      774
floor_count    1094
dtype: int64

In [82]:
#Number of unique values in each column
building.nunique()

site_id          16
building_id    1449
primary_use      16
square_feet    1397
year_built      116
floor_count      18
dtype: int64

In [83]:
building['year_built'].value_counts()

1976.0    55
1966.0    23
1968.0    18
1919.0    17
1964.0    15
2004.0    14
1960.0    13
1975.0    13
2007.0    13
2006.0    13
2014.0    12
2010.0    12
1970.0    12
2001.0    12
2002.0    11
1930.0    10
1967.0    10
2005.0    10
1959.0    10
1989.0     9
2013.0     9
1956.0     8
1999.0     8
1923.0     8
1969.0     7
1990.0     7
2016.0     7
2011.0     7
1963.0     7
1958.0     7
1981.0     6
1913.0     6
1996.0     6
1953.0     6
1974.0     6
1912.0     6
1932.0     6
1965.0     6
1931.0     6
1962.0     5
2003.0     5
1948.0     5
1940.0     5
1986.0     5
2000.0     5
1909.0     5
2008.0     5
1942.0     5
1957.0     5
1910.0     5
1978.0     5
1971.0     5
1995.0     5
1955.0     5
1941.0     5
1900.0     5
1961.0     5
1951.0     5
1950.0     4
1914.0     4
1929.0     4
1935.0     4
1945.0     4
1985.0     4
1982.0     4
1933.0     4
1980.0     4
1993.0     4
2009.0     4
1997.0     4
1979.0     4
1949.0     4
1911.0     4
2012.0     4
1991.0     4
1983.0     4
1977.0     3

In [84]:
building.describe()

,site_id,building_id,square_feet,year_built,floor_count
count,1449.000000,1449.000000,1449.000000,675.000000,355.000000
mean,6.952381,724.000000,92111.776398,1967.957037,3.740845
std,5.003432,418.434583,110769.950997,31.054030,3.333683
min,0.000000,0.000000,283.000000,1900.000000,1.000000
25%,3.000000,362.000000,23012.000000,1949.000000,1.000000
50%,5.000000,724.000000,57673.000000,1970.000000,3.000000
75%,13.000000,1086.000000,115676.000000,1995.000000,5.000000
max,15.000000,1448.000000,875000.000000,2017.000000,26.000000


#### The way the null values of the year_built column is, i have to separate it into bins

In [85]:
#due to the unneven and complex nature of the missing values in the year column, interpolation is the best solution
building['year_built']=building['year_built'].interpolate()

In [87]:
building['year_built'].value_counts()

1976.000000    57
1966.000000    25
1968.000000    19
1919.000000    17
1964.000000    16
2004.000000    14
1960.000000    14
1975.000000    14
2006.000000    13
1970.000000    13
2014.000000    13
2007.000000    13
2010.000000    12
1930.000000    12
2001.000000    12
2002.000000    11
2005.000000    11
1959.000000    11
2013.000000    11
1989.000000    10
1967.000000    10
1990.000000     9
1999.000000     8
1923.000000     8
1981.000000     8
1978.000000     8
1931.000000     8
1942.000000     8
1956.000000     8
1958.000000     8
1963.000000     8
1969.000000     8
1986.000000     7
2011.000000     7
1955.000000     7
1953.000000     7
1980.000000     7
2016.000000     7
1974.000000     7
1940.000000     6
1993.000000     6
2000.000000     6
1913.000000     6
1951.000000     6
1932.000000     6
1995.000000     6
1961.000000     6
2008.000000     6
1971.000000     6
1996.000000     6
1912.000000     6
1985.000000     6
2003.000000     6
1965.000000     6
1939.000000     5
1957.00000

In [88]:
building['year_built'].isnull().sum()

0

In [89]:
building['floor_count'].isnull().sum()

1094

###### There are many too missing values here and the number of floors in a building does not have any effect on energy consumption so column would be removed

In [90]:
building=building.drop(columns='floor_count')

In [91]:
building.head(1)

,site_id,building_id,primary_use,square_feet,year_built
0,0,0,Education,7432,2008.0


In [92]:
#check for missing values in the weather data
weather.isnull().sum()

site_id                   0
timestamp                 0
air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
dtype: int64

In [93]:
weather.describe()

,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
count,139773.000000,139718.000000,70600.000000,139660.000000,89484.000000,129155.000000,133505.000000,139469.000000
mean,7.478977,14.418106,2.149306,7.350158,0.983047,1016.158038,180.526632,3.560527
std,4.604744,10.626595,2.599150,9.790235,8.463678,7.629684,111.523629,2.335874
min,0.000000,-28.900000,0.000000,-35.000000,-1.000000,968.200000,0.000000,0.000000
25%,3.000000,7.200000,0.000000,0.600000,0.000000,1011.800000,80.000000,2.100000
50%,7.000000,15.000000,2.000000,8.300000,0.000000,1016.400000,190.000000,3.100000
75%,11.000000,22.200000,4.000000,14.400000,0.000000,1020.800000,280.000000,5.000000
max,15.000000,47.200000,9.000000,26.100000,343.000000,1045.500000,360.000000,19.000000


In [94]:
weather['air_temperature'].value_counts()

 15.0    2722
 24.4    2200
 20.0    2191
 10.0    2187
 25.0    2185
 19.4    2121
 23.9    2115
 25.6    2096
 17.2    2095
 23.3    2074
 18.3    2067
 15.6    2066
 20.6    2043
 18.9    2014
 14.4    2010
 17.8    2001
 22.8    1989
 13.9    1987
 21.7    1971
 21.1    1947
 16.7    1936
 26.1    1915
 13.3    1893
 22.2    1881
 16.1    1862
 26.7    1798
 12.8    1680
 12.2    1679
 11.7    1622
 27.2    1616
 11.1    1512
 27.8    1463
 10.6    1455
 9.4     1396
 5.0     1393
 28.3    1372
 28.9    1271
 8.3     1267
 8.9     1247
 7.8     1213
 6.7     1192
 7.2     1176
 6.1     1147
 29.4    1121
 30.0    1104
 5.6     1095
 4.4     1000
 2.8      985
 30.6     951
 3.9      933
 31.1     917
 3.3      911
 0.6      892
 2.2      857
 1.1      852
 31.7     827
 1.7      823
 8.0      815
 0.0      812
 6.0      764
 7.0      736
-0.6      730
 16.0     728
 32.2     710
 9.0      697
 14.0     682
-1.1      675
 32.8     674
 11.0     673
 13.0     647
 12.0     623
 17.0 

In [95]:
#drop the rows with missing data in the air temperature column
weather=weather.dropna(subset=['air_temperature'])

In [96]:
#check to see if missing values have been dropped
weather['air_temperature'].isnull().sum()

0

In [97]:
#drop the rows with missing data in the dew temperature column
weather=weather.dropna(subset=['dew_temperature'])

In [98]:
#check to see if missing values have been dropped
weather['dew_temperature'].isnull().sum()

0

In [99]:
weather['wind_speed'].value_counts()

2.1     14371
2.6     14043
0.0     13538
1.5     13087
3.1     12961
3.6     11878
4.1      9977
4.6      8261
5.1      6632
5.7      5412
6.2      4456
6.7      3522
7.2      2774
1.0      2056
7.7      1979
8.2      1497
4.0      1413
5.0      1358
6.0      1152
3.0      1133
0.5      1059
8.8      1019
7.0       861
9.3       730
2.0       669
8.0       604
9.8       495
9.0       449
10.3      320
10.0      305
10.8      271
11.0      207
11.3      190
11.8      122
12.0      119
12.4       81
13.0       70
12.9       57
8.7        57
14.0       36
13.4       36
13.9       26
14.4       23
15.0       18
16.0       13
15.4       13
14.9       12
17.0        5
16.5        4
19.0        3
1.3         3
2.2         3
18.0        3
18.5        2
3.2         2
1.6         2
6.1         2
12.8        2
Name: wind_speed, dtype: int64

In [100]:
#drop the rows with missing data in the wind speed column
weather=weather.dropna(subset=['wind_speed'])

In [101]:
#check to see if missing values have been dropped
weather['wind_speed'].isnull().sum()

0

##### Regarding the cloud_coverage,precip_depth_1_hr,sea_level_pressure and wind direction,  the missing values would have to be interpolated

In [102]:
weather.isnull().sum()

site_id                   0
timestamp                 0
air_temperature           0
cloud_coverage        68947
dew_temperature           0
precip_depth_1_hr     50229
sea_level_pressure    10549
wind_direction         5965
wind_speed                0
dtype: int64

In [103]:
weather=weather.interpolate()

In [104]:
#one more missing value due to interpolation not counting first values,and that particular missing data was the first
weather.isnull().sum()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     1
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [105]:
weather['precip_depth_1_hr']=weather['precip_depth_1_hr'].fillna(weather['precip_depth_1_hr'].mode())

In [106]:
weather.isnull().sum()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [107]:
data['meter'].value_counts()

0    12060910
1     4182440
2     2708713
3     1264037
Name: meter, dtype: int64

In [108]:
data.isnull().sum()

building_id      0
meter            0
timestamp        0
meter_reading    0
dtype: int64

##### Now all the missing values in all the datasets have been dealt with, maybe there are better ways of dealing with this, but we are going to go with this 

In [109]:
data.to_csv('data_new')
weather.to_csv('weather_new')
building.to_csv('building_new')